In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import time
import os
import pickle

from my_lib.utility import *

## Actividad 3: Algoritmo genético para resolver el problema del viajero frecuente

El problema es de tipo TSP con 18 ciudades del conteninte Americano. Por lo tanto existen $18!\$ soluciones posibles ($6402373705728000$), dada el basto espacio de busqueda la población de individuos se debe limitar a minimo 100 y máximo 500.

> Criterios para la evaluación
> - Implementar 2 métodos de cruza, PMX y algún otro método como OX o CX
> - Implementar 2 métodos de mutación, Scramble y Heuristic
> - Implementar un método de selección, cualquiera de los ya vistos
> - Implementar un método hibrido de paro
> - Implementar un méotodo elitista
> - Realizar las comparaciones y la discusión de los resultados

# Codificación del dominio

In [11]:
ciudades18 = {
    'Ciudad': ['Mexico City', 'Merida', 'Buenos Aires', 'Quito', 'Washington D.C.', 
               'New York', 'Miami', 'Monterrey', 'Panama City', 'San Salvador',
               'Managua', 'Brazilia', 'Mendoza', 'Caracas', 'Montevideo', 
               'Guadalajara', 'Boston', 'Bogotá'],
    'Latitud': [19.4326, 20.9674, -34.6037, -0.1807, 38.9072, 
                40.7128, 25.7617, 25.6866, 8.9824, 13.6929, 
                12.1140, -15.8267, -32.8895, 10.4806, -34.9011, 
                20.6597, 42.3601, 4.7110],
    'Longitud': [-99.1332, -89.5926, -58.3816, -78.4678, -77.0369, 
                 -74.0060, -80.1918, -100.3161, -79.5199, -89.2182, 
                 -86.2362, -47.9218, -68.8458, -66.9036, -56.1645, 
                 -103.3496, -71.0589, -74.0721]
} 
nombre_ciudades = np.array(ciudades18['Ciudad'])

In [12]:
id_ciudades = np.arange(nombre_ciudades.shape[0])
bin_ciudades = binarizar(datos=id_ciudades,nbites=3)

ciudades = np.zeros((nombre_ciudades.shape[0],4),dtype="object")
ciudades[:,0] = nombre_ciudades
ciudades[:,1] = id_ciudades
ciudades[:,2] = bin_ciudades
ciudades[:,3] = np.zeros((nombre_ciudades.shape[0]))
print(ciudades)

[['Mexico City' 0 '000' 0.0]
 ['Merida' 1 '001' 0.0]
 ['Buenos Aires' 2 '010' 0.0]
 ['Quito' 3 '011' 0.0]
 ['Washington D.C.' 4 '100' 0.0]
 ['New York' 5 '101' 0.0]
 ['Miami' 6 '110' 0.0]
 ['Monterrey' 7 '111' 0.0]
 ['Panama City' 8 '1000' 0.0]
 ['San Salvador' 9 '1001' 0.0]
 ['Managua' 10 '1010' 0.0]
 ['Brazilia' 11 '1011' 0.0]
 ['Mendoza' 12 '1100' 0.0]
 ['Caracas' 13 '1101' 0.0]
 ['Montevideo' 14 '1110' 0.0]
 ['Guadalajara' 15 '1111' 0.0]
 ['Boston' 16 '10000' 0.0]
 ['Bogotá' 17 '10001' 0.0]]


## Matriz de distancias

#### Fórmula de Haversine 

$d = 2r \cdot arsin(\sqrt{sin^2(\frac{\Delta\phi}{2}) + cos(\phi_1) \cdot cos(\phi_2) \cdot sin^2(\frac{\Delta\lambda}{2})})$

* d es la distancia entre los dos puntos 
* r = radio de la tierra (6371)
* $\phi_1, \phi_2$ = latitudes de los puntos en rad
* $\Delta\phi$ = diferencia de las latitudes
* $\Delta\lambda$ = diferencia de las longitudes


In [13]:
# Latitud y longitud de cada ciudad

n_ciudades = ciudades.shape[0]
distancias = np.zeros((n_ciudades, n_ciudades))

for i in range(n_ciudades):
    c1_latitud = ciudades18["Latitud"][i]
    c1_longitud = ciudades18["Longitud"][i]
    for j in range(n_ciudades):
        if i != j:
            c2_latitud = ciudades18["Latitud"][j]
            c2_longitud = ciudades18["Longitud"][j]
            distancias[i][j] = get_distancia(
                lat1=c1_latitud, long1=c1_longitud, lat2=c2_latitud, long2=c2_longitud
            )

distancias_df = pd.DataFrame(distancias,columns=ciudades18["Ciudad"],index=ciudades18["Ciudad"])
display(distancias_df)

,Mexico City,Merida,Buenos Aires,Quito,Washington D.C.,New York,Miami,Monterrey,Panama City,San Salvador,Managua,Brazilia,Mendoza,Caracas,Montevideo,Guadalajara,Boston,Bogotá
Mexico City,0.000000,1009.959843,7392.178684,3135.709015,3031.811604,3359.383041,2065.272960,705.925938,2408.857647,1233.956557,1600.957943,6835.065741,6648.609671,3595.449156,7554.599238,461.067374,3665.482788,3170.731318
Merida,1009.959843,0.000000,7004.211339,2644.150812,2327.565260,2643.050506,1097.212425,1213.520720,1715.030837,809.861482,1047.292813,6116.463263,6381.173335,2689.070020,7144.536815,1429.853511,2941.198434,2464.943628
Buenos Aires,7392.178684,7004.211339,0.000000,4360.060643,8396.527908,8526.194114,7094.952486,8028.590573,5333.037977,6280.332978,5968.507501,2334.403612,985.614665,5093.044534,205.232076,7766.608993,8654.780454,4670.520317
Quito,3135.709015,2644.150812,4360.060643,0.000000,4348.827501,4569.460001,2890.594533,3711.248568,1025.530802,1944.679440,1613.735904,3776.161246,3774.047997,1743.741681,4501.636863,3562.151769,4788.167318,730.923189
Washington D.C.,3031.811604,2327.565260,8396.527908,4348.827501,0.000000,327.582714,1491.176569,2623.500270,3336.690529,3048.362836,3114.733025,6797.691847,8028.085133,3317.116974,8483.304262,3229.236038,633.671486,3814.195109
New York,3359.383041,2643.050506,8526.194114,4569.460001,327.582714,0.000000,1757.961225,2945.076096,3570.004517,3348.328915,3397.166346,6839.011011,8201.327390,3433.343519,8603.779701,3554.283691,306.108494,4003.222959
Miami,2065.272960,1097.212425,7094.952486,2890.594533,1491.176569,1757.961225,0.000000,2013.994655,1867.122418,1639.742840,1644.533412,5794.758090,6631.402300,2200.658515,7204.245436,2430.007251,2025.336834,2429.837957
Monterrey,705.925938,1213.520720,8028.590573,3711.248568,2623.500270,2945.076096,2013.994655,0.000000,2876.282439,1766.700637,2111.060860,7310.080266,7315.684383,3898.681265,8184.245928,639.142332,3245.755067,3640.297547
Panama City,2408.857647,1715.030837,5333.037977,1025.530802,3336.690529,3570.004517,1867.122418,2876.282439,0.000000,1179.637483,812.486225,4440.058794,4791.336430,1392.559785,5462.750288,2865.971802,3802.437689,766.254430
San Salvador,1233.956557,809.861482,6280.332978,1944.679440,3048.362836,3348.328915,1639.742840,1766.700637,1179.637483,0.000000,367.803223,5601.769099,5613.686901,2451.382655,6430.418383,1687.991760,3634.269046,1937.742276


# Generación de población

Para poder realizar una buena comparación con las distintas combinaciones de operadores genéticos se trabajará con 5 poblaciones iniciales, las cuales son las mismas parar cada experimento


Si ya existe el archivo de poblaciones no hay necesidad de crear nuevamente las poblaciones iniciales

In [14]:
POB_FILE = "files\poblaciones_3.pkl"

In [15]:
def get_poblaciones_iniciales(n=5, tam_pob=60):
    """
    Genera y carga las poblaciones iniciales para un algoritmo genético.

    Si el archivo de poblaciones no existe, la función lo crea generando las poblaciones iniciales y guardándolas en el archivo.
    Si el archivo ya existe, la función lo carga y devuelve las poblaciones.

    Parámetros:
        n (int): Número de poblaciones iniciales a generar (por defecto, 5).
        tam_pob (int): Tamaño de cada población (por defecto, 60).

    Valor de retorno:
        list: Lista de poblaciones iniciales, donde cada población es una matriz de numpy con 4 columnas: ruta, aptitud, individuo y aptitud.
    """
    poblaciones = []
    POB_PATH = os.path.join("..", POB_FILE)
    if not os.path.exists(POB_PATH):
        print("El archivo de poblaciones no existse, creando archivo")
        
        for i in range(n):
            recorrido = generar_poblacion_perm(ciudades, tam_pob)
            
            poblacion = np.zeros((recorrido.shape[0], 4), dtype="object")
            poblacion[:, 0] = [c.astype(int) for c in recorrido]  # Ruta
            poblacion[:, 1] = np.ones((recorrido.shape[0]))
            poblacion[:, 2] = [individuo_toString(c.astype(int)) for c in recorrido]
            poblacion[:, 3] = np.zeros((recorrido.shape[0]))  # Aptitud
        
            poblaciones.append(poblacion)
        
        try:
            with open(POB_PATH, "wb") as f:
                pickle.dump(poblaciones,f)
            print("Archivo de poblaciones creado correctamente")
        except Exception as e:
            print(f"Error al guardar archivo: {e}")
            
    else:
        try:
            with open(POB_PATH, "rb") as f:
                poblaciones = pickle.load(f)
            print("Archivo de poblaciones cargado correctamente")
        except Exception as e:
            print(f"Error al cargar archivo: {e}")
    
    return poblaciones

In [18]:
pobs = get_poblaciones_iniciales(n=5,tam_pob=100)
print(len(pobs))
print(pobs[0][:2])

Archivo de poblaciones cargado correctamente
5
[[array([ 5, 13,  0, 12,  3,  4,  6,  9, 17, 10,  8, 11, 15,  7, 14, 16,  2,
          1])
  1.0 ' 513 012 3 4 6 91710 81115 71416 2 1' 0.0]
 [array([11, 17,  0,  2, 12, 16, 10,  9,  7, 15,  4, 14,  6,  8,  1,  3, 13,
          5])
  1.0 '1117 0 2121610 9 715 414 6 8 1 313 5' 0.0]]


# Función objetivo

In [19]:
def y(ciudad1,ciudad2):
    return distancias[ciudad1,ciudad2]

def evaluar(poblacion):
    evaluacion = np.zeros((poblacion.shape[0]))
    for idx,fila in enumerate(poblacion):
        individuo = fila[0]
        n = len(individuo) - 1
        aptitud = 0
        
        for i in range (n,-1,-1):
            c1 = int(individuo[i]) #Id de la ciudad
            c2 = int(individuo[i-1])
            distancia = y(c1,c2)
            aptitud += distancia
           # print(f"Distancia entre {ciudades[c1]} y {ciudades[c2]}: {distancia}")
        evaluacion[idx] = np.around(aptitud,4)
        
    return evaluacion



# Experimento combinacion PMX, y Heuristic mutation

En este experimento a partir de la población incial, tomar en cuenta que hay 5 poblaciones inciales con las cuales debe realizarse el experimento, se obtienen nuevas generaciones de individuos y una vez alcanza el 80% de individuos aptos, esto quiere decir que los individuso deben superar más del 0.8 de aptitud, o se llega un 10 generaciones el algoritmo se detiene.

**Condiciones de termino:**

* Epsilon:
    * Cuando el 80% de la población alcanza una aptitud mayor de 0.8
* Generaciones:
    * Cuando se llega a la generación número 10

**Operadores:**
* Método de competencia genética para la creación de las poblaciones
* Operador de selección *Ruleta*
* Operador de cruce *PMX*
* Operador de mutación  *Hueristico*

## Evaluar y Ordenar la poblacion

La función de ordenar población ordena de mayor a menor.

Este problema es un problema para encontrar las rutas con menor costo

In [31]:
MAX_GEN = 10

In [32]:
#Tiempo del experimento total
start_exp_time = time.time()
for i in range(len(pobs)): 
    #Cargar población incial
    print(f"Población inicial: {i}")
    poblacion = pobs[i]
    #evaluar la población
    pob_aptitud = evaluar(poblacion)
    poblacion[:,3] = pob_aptitud
    #La población esta ordenada de mayor a menor, por lo tanto los últimos indiviudos siempre son los más aptos 
    poblacion = ordenar_poblacion(poblacion,reverse=True)
    
    # print(poblacion.shape)
    # print(poblacion[-10:])
    
    #Experimentación
    GEN = 0
    GENERACIONES = []
    HISTDF = []
     
    GENERACIONES.append(poblacion)
    
    start_time = time.time()
    while True:
        #poblacion actual
        print(f"Generación: {GEN}")
        print(np.mean(poblacion[:,3]))
        
        if GEN >= MAX_GEN or paro_epsilon(poblacion,14000,0.9,opt=1):
            print("Convergencia alcanzada")
            break
        
        else:
            
            #Obtener siguiente generacion
            #Selección y cruza
            hijos = get_next_gen(poblacion,op_seleccion=0,op_cruza=2,mode="min")
            print(hijos)
            
            GEN += 1
            
            
            
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Tiempo de ejecucion poblacion {i} : {np.around(elapsed_time)} segundos")
    break


end_exp_time = time.time()
elapsed_exp_time = end_exp_time - start_exp_time

Población inicial: 0
Generación: 0
69168.40171100001
[[array([ 2,  3, 11, 10, 14, 12,  7, 16, 15,  5,  4, 17,  1,  8,  0, 13,  9,
        6]), array([' 2 31110141216 4 515 717 1 8 013 9 6',
       ' 014 811 9 5 7161517 1 61013 212 4 3'], dtype=object)], [array([ 0, 14,  8, 11,  9, 15, 16,  4,  5, 17,  1,  6, 10, 13,  2, 12,  7,
        3]), array([' 2 31110141216 4 515 717 1 8 013 9 6',
       ' 014 811 9 5 7161517 1 61013 212 4 3'], dtype=object)], [array([17, 14, 10, 15, 12,  3,  7,  8,  4,  9,  6, 11,  2,  0, 16,  1, 13,
        5]), array(['171410 6 8 0 7 912 41511 2 316 113 5',
       ' 5 1 01512 3 7 8 41610141713 2 611 9'], dtype=object)], [array([ 5,  1,  3,  6,  8,  0,  7,  9, 12, 16, 10, 14, 17, 13,  2, 15, 11,
        4]), array(['171410 6 8 0 7 912 41511 2 316 113 5',
       ' 5 1 01512 3 7 8 41610141713 2 611 9'], dtype=object)], [array([14, 16,  6,  1,  7, 10, 12,  4, 11,  5,  3, 13,  8, 15,  9,  0,  2,
       17]), array([' 71112151410 0 8 317 513 4 1 9 6 216',
       '15

In [33]:
print(f"Tiempo de experimentación: {elapsed_exp_time}")

Tiempo de experimentación: 0.723508358001709
